# Datensatz splitten

In [1]:
import os
import random
import shutil
from glob import glob
import yaml

---

## Einstellungen

In [2]:
# Split-Verhältnis einstellen
SPLIT_RATIOS = {
    'train': 0.7,      # 70% für Training
    'val': 0.2,        # 20% für Validierung
    'test': 0.1        # 10% für Test
}

AUG_IMAGES_DIR = r"C:\Users\jonni\OneDrive\Dokumente\GitHub\Chairlift_Gefahrenerkennung\01_Data_Jon\Augmented\Images"
AUG_LABELS_DIR = r"C:\Users\jonni\OneDrive\Dokumente\GitHub\Chairlift_Gefahrenerkennung\01_Data_Jon\Augmented\Labels"

# (Optional: Anpassen/ergänzen)
CLASS_NAMES = [
    'KLEBER', 'EDDING', 'TACKER'  # Ersetze/ergänze mit deinen tatsächlichen Klassen
]


In [3]:

# Name des Quellordners der Bilder extrahieren
SOURCE_DIR = os.path.basename(AUG_IMAGES_DIR)

# Verzeichnis in dem das Bild Verzeichnis liegt
SOURCE_DIR_PATH = os.path.dirname(AUG_IMAGES_DIR)

# Zielverzeichnis für die Splits ist der gleiche wie das Quellverzeichnis.
# Der Name des Quellverzeichnisses wird als Zielverzeichnis verwendet und mit "_split" ergänzt.
TARGET_DIR = os.path.join(SOURCE_DIR_PATH, f"{SOURCE_DIR}_split")
os.makedirs(TARGET_DIR, exist_ok=True)

# Zielverzeichnis für die Splits
DATASET_ROOT = os.path.join(TARGET_DIR, SOURCE_DIR)
os.makedirs(DATASET_ROOT, exist_ok=True)


---

### Split Vorgang

In [4]:
# Split-Verzeichnisse erstellen
def make_dirs():
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(DATASET_ROOT, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(DATASET_ROOT, split, 'labels'), exist_ok=True)

make_dirs()

# Alle Bilder finden (nur augmentierte verwenden!)
all_images = sorted(glob(os.path.join(AUG_IMAGES_DIR, '*.jpg')) + glob(os.path.join(AUG_IMAGES_DIR, '*.png')))
random.shuffle(all_images)

n_total = len(all_images)
n_train = int(SPLIT_RATIOS['train'] * n_total)
n_val = int(SPLIT_RATIOS['val'] * n_total)
n_test = n_total - n_train - n_val

splits = {
    'train': all_images[:n_train],
    'val': all_images[n_train:n_train+n_val],
    'test': all_images[n_train+n_val:]
}

for split, files in splits.items():
    for img_path in files:
        base = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(AUG_LABELS_DIR, f"{base}.txt")
        # Zielpfade
        out_img = os.path.join(DATASET_ROOT, split, 'images', f"{base}.jpg")
        out_label = os.path.join(DATASET_ROOT, split, 'labels', f"{base}.txt")
        # Kopieren
        shutil.copy(img_path, out_img)
        shutil.copy(label_path, out_label)

print(f"Datensatz gesplittet: {n_train} train, {n_val} val, {n_test} test")

# dataset.yaml erzeugen
yaml_dict = {
    'train': os.path.abspath(os.path.join(DATASET_ROOT, 'train', 'images')),
    'val': os.path.abspath(os.path.join(DATASET_ROOT, 'val', 'images')),
    'test': os.path.abspath(os.path.join(DATASET_ROOT, 'test', 'images')),
    'nc': len(CLASS_NAMES),
    'names': CLASS_NAMES
}
with open(os.path.join(DATASET_ROOT, 'dataset.yaml'), 'w') as f:
    yaml.dump(yaml_dict, f, sort_keys=False, allow_unicode=True)

print("dataset.yaml erstellt!")
print(f"Gespeichert in: {os.path.abspath(os.path.join(DATASET_ROOT, 'dataset.yaml'))}")


Datensatz gesplittet: 315 train, 90 val, 45 test
dataset.yaml erstellt!
Gespeichert in: C:\Users\jonni\OneDrive\Dokumente\GitHub\Chairlift_Gefahrenerkennung\01_Data_Jon\Augmented\Images_split\Images\dataset.yaml
